In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
!pip install roboflow
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="ZlV3OwtaEo7oCbxTsZaj")
project = rf.workspace("eyet-apkje").project("eyet-ycrjy")
dataset = project.version(1).download("yolov5")

In [ ]:
import os

# 라벨 폴더 경로 지정 (train, valid, test 모두 처리)
label_dirs = [
    '/content/EyeT-1/train/labels',
    '/content/EyeT-1/test/labels',
    '/content/EyeT-1/valid/labels'
]

# 클래스 번호 매핑 (대소문자 중복 통합)
# 예: ESOTROPIA(0) + esotropia(5) -> 0
mapping = {
    5: 0,  # esotropia → ESOTROPIA
    6: 1,  # exotropia → EXOTROPIA
    7: 2,  # hypertropia → HYPERTROPIA
    8: 3,  # hypotropia → HYPOTROPIA
    9: 4   # normal → NORMAL
}

def merge_labels(label_dir):
    if not os.path.exists(label_dir):
        print(f"❌ 경로 없음: {label_dir}")
        return

    for file_name in os.listdir(label_dir):
        if not file_name.endswith('.txt'):
            continue

        file_path = os.path.join(label_dir, file_name)
        new_lines = []

        with open(file_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue

            cls = int(parts[0])
            # 클래스 매핑 적용
            if cls in mapping:
                cls = mapping[cls]
            parts[0] = str(cls)
            new_lines.append(' '.join(parts))

        with open(file_path, 'w') as f:
            f.write('\n'.join(new_lines))

        print(f"✅ 수정됨: {file_name}")

# 실행
for d in label_dirs:
    merge_labels(d)

print("\n🎉 클래스 통합 완료! 이제 data.yaml의 names도 5개만 남기면 돼:")
print("['ESOTROPIA', 'EXOTROPIA', 'HYPERTROPIA', 'HYPOTROPIA', 'NORMAL']")


In [ ]:
# ✅ 3. 데이터셋 경로 연결
# Roboflow에서 받은 dataset이 /content/dataset 형태로 있다면:
data_path = "/content/EyeT-1/data.yaml"
!python /content/yolov5/train.py --img 640 --batch 16 --epochs 100 --data {data_path} --weights yolov5s.pt --project "/content/drive/MyDrive/EyeT_project/runs" --name eyet_model --save-period 5

In [ ]:
# Colab 셀에 복사하여 실행하세요

# 1. 웹 서버 및 WebSocket용 라이브러리
!pip install "fastapi[all]" python-socketio

# 2. Colab 외부 접속을 위한 ngrok
!pip install pyngrok

# 3. 이미지 처리를 위한 OpenCV 및 YOLOv5 의존성
!pip install opencv-python-headless numpy pillow

In [ ]:
# Colab 셀에 복사하여 실행하세요

import torch
import fastapi
import uvicorn
import socketio
import numpy as np
import cv2
import base64
import io
from PIL import Image
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware

# --- 1. ngrok 및 기본 설정 ---

# 🔴 중요: 여기에 본인의 ngrok 인증 토큰을 붙여넣으세요.
# ( https://dashboard.ngrok.com/get-started/your-authtoken )
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# FastAPI 및 Socket.IO 서버 앱 생성
app = fastapi.FastAPI()
sio = socketio.AsyncServer(async_mode="asgi", cors_allowed_origins="*")
socket_app = socketio.ASGIApp(sio, other_asgi_app=app)

# CORS 설정 (모든 출처 허용)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# --- 2. YOLOv5 모델 로드 ---

# 훈련된 best.pt 모델 경로
MODEL_PATH = "/content/drive/MyDrive/EyeT_project/runs/eyet_model7/weights/best.pt"

# YOLOv5 레포지토리에서 모델 구조 로드
# 'custom'을 사용하고, 훈련된 가중치(path)를 지정합니다.
try:
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_PATH, force_reload=True)
    model.conf = 0.25  # 신뢰도 임계값 설정 (필요에 따라 조절)
    print("✅ YOLOv5 모델 로드 성공.")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("Colab 런타임에 'yolov5' 폴더가 있는지 확인하세요.")
    print("만약 없다면, !git clone https://github.com/ultralytics/yolov5 실행 후 이 셀을 다시 시작하세요.")


# --- 3. WebSocket 이벤트 핸들러 ---

@sio.on("connect")
async def connect(sid, environ):
    print(f"✅ 클라이언트 연결됨: {sid}")

@sio.on("disconnect")
async def disconnect(sid):
    print(f"❌ 클라이언트 연결 끊김: {sid}")

@sio.on("image")  # 프론트엔드에서 "image" 이벤트로 데이터를 보낼 것임
async def handle_image(sid, data):
    """
    프론트엔드로부터 base64 인코딩된 이미지 수신,
    YOLOv5 추론[Flowchart C] 후 결과 반환[Flowchart D]
    """
    try:
        # base64 이미지 디코딩 (data: "data:image/jpeg;base64,...")
        img_data = base64.b64decode(data.split(',')[1])

        # 바이너리 데이터를 OpenCV 이미지로 변환
        pil_image = Image.open(io.BytesIO(img_data))
        frame = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

        # YOLOv5 추론 수행
        results = model(frame)

        # 추론 결과를 pandas DataFrame으로 변환 후 JSON으로 직렬화
        results_df = results.pandas().xyxy[0]  # [xmin, ymin, xmax, ymax, confidence, class, name]
        results_json = results_df.to_dict(orient="records")

        # 프론트엔드로 "results" 이벤트 발송
        await sio.emit("results", results_json, to=sid)

    except Exception as e:
        print(f"이미지 처리 오류: {e}")
        await sio.emit("error", str(e), to=sid)


# --- 4. ngrok 터널링 및 서버 실행 ---
@app.get("/")
def read_root():
    return {"Status": "YOLOv5 Strabismus Detection Server is Running"}

# ngrok으로 public URL 생성
http_tunnel = ngrok.connect(8000, "http")
print("=================================================================")
print(f"✅ 서버가 실행 중입니다.")
print(f"📡 프론트엔드에서 접속할 WebSocket 주소:")
print(f"   {http_tunnel.public_url}")
print("=================================================================")

import nest_asyncio
import asyncio

# 1. nest_asyncio를 적용하여 루프 중첨 허용
nest_asyncio.apply()

config = uvicorn.Config(
    socket_app,
    host="0.0.0.0",
    port=8000,
    log_level="info"
)
server = uvicorn.Server(config)

loop = asyncio.get_event_loop()
loop.run_until_complete(server.serve())